 # <center> Projeto Web Scraping : Extração de Dados de Passagens Aéreas </center>

# *Sumário*
<font size=3>
    <ul>
       <li> O que é Web Scraping?</li>
       <li> Sobre o projeto.</li>
       <li> Importando as bibliotecas.</li>
        <li>Utilizando o Selenium.</li> 
    </ul>
</font>

------------------------------------------------------------------

### *O que é Web Scrapping?*

<p><b>Web scraping</b> é uma coleta de dados da web, de sites, onde são usados scripts e programas para “raspar” informações destes sites e que poderão ser usadas para futuras análises. Por exemplo, comparar preços, monitorar o tempo, pesquisar o que está em alta no mercado, entre muitas outras coisas.</p>

------------------------------------------------------------------

### *Sobre o projeto:*

<p>Recentemente venho procurado de forma exaustiva passagens aéreas para Buenos Aires(Argentina), afim de encontrar os melhores preços. Percebendo o trabalho que seria, tive a ideia de colocar em prática um assunto que chamou muito minha atenção, o Web Scraping, assim escolhi o site da <a href='https://www.decolar.com'>Decolar</a> para "raspar" os dados e obter as melhores promoções. A ideia é gerar um script onde com apenas um click eu consiga extrair os dados necessários para fazer uma boa análise.</p>

<center><img src='https://vemvoar.voeazul.com.br/wp-content/uploads/2018/08/as-10-rotas-de-aviao-mais-longas-do-mundo.jpeg' alt='Plane' width='450' height='100'></center>

------------------------------------------------------------------

## *Importando as bibliotecas:*

In [1]:
#urllib
from urllib.request import urlopen
from urllib.request import urlretrieve

#BeautifulSoup
from bs4 import BeautifulSoup

#Pandas
import pandas as pd

------------------------------------------------------------------

### Utilizando o Selenium:

<p>Algumas páginas são renderizadas via <code>Javascrip</code> que é carregado dinamicamente, por isso utilizamos a biblioteca Selenium para contornar esse problema.</p>

In [164]:
%%time
#Selenium
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chromium import service
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

    
#Definindo o tamanho da janela do navegador
options = Options()
#options.add_argument('--headless')
options.add_argument("--start-maximized")


#Acessando a página
url = 'https://www.decolar.com/passagens-aereas/SAO/BUE?from=SB&di=1-0'
navegador = webdriver.Chrome()
navegador.get(url)
sleep(2)

#Percorrendo a página

body = navegador.find_element_by_css_selector('body')
body.send_keys(Keys.END)


for i in range(0,12):
    body = navegador.find_element_by_css_selector('body')
    body.send_keys(Keys.END)
    sleep(0.5)
    WebDriverWait(navegador, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="clusters"]/div/a/em'))).click()



sleep(2)

soup = BeautifulSoup(navegador.page_source, 'html.parser')

<timed exec>:27: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
<timed exec>:32: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead


CPU times: user 1.93 s, sys: 78 ms, total: 2 s
Wall time: 24.2 s


### Obtendo as informações:

In [165]:
#cluster-container border not-overflow
voos = soup.findAll('div', {'class':'cluster-container border not-overflow'})
cards = []

for voo in voos:
    card = {}
    valor = voo.find('span', {'class':'pricebox-big-text price'}).get_text()
    valor = float(valor) * 1000
    card['Valor'] = valor
    
    #Obtendo o nome da Companhia Aérea
    nome = voo.find('span', {'class':'name'}).get_text().strip()
    card['Companhia Aérea'] = nome

    #Horário do Voô
    horario = voo.find('span', {'class':"stops-text text -eva-3-tc-gray-2"}).get_text()[:-1]
    card['Horário de Voo'] = horario

    #Se ele é direto ou não
    direto = voo.find('span', {'class':'stops-text text -eva-3-tc-green-3'}).get_text().strip()
    #card['É Direto'] = direto

    #Dia da Semana Ida
    dia_da_semana_ida = voo.find('div', {'class':'date -eva-3-bold route-info-item-date lowercase'}).get_text().strip()[:3]
    card['Dia da Semana Ida'] = dia_da_semana_ida


    #Dias de Viagem
    dias = voo.find('span', {'class':'elipsis-days primary-message'}).get_text().strip().split()[0]
    dias = int(dias)
    card['Dias de Viagem'] = dias
    
    #Data IDA
    data_ida = voo.find('div', {'class':'date -eva-3-bold route-info-item-date lowercase'}).get_text()[5:].replace('.','')
    card['Data Ida'] = data_ida
    
    cards.append(card)

#Criando o DataFrame com as informações
teste = pd.DataFrame(cards)
teste.shape

(150, 6)

In [166]:
#Salvando o DataFrame em um arquivo CSV
teste.to_csv('voos_testes.csv', index = False)

In [167]:
teste.head()

,Valor,Companhia Aérea,Horário de Voo,Dia da Semana Ida,Dias de Viagem,Data Ida
0,1462.0,Ethiopian Airlines,17:00,Seg,9,21 Nov 2022
1,1462.0,Ethiopian Airlines,17:00,Sáb,4,26 Nov 2022
2,1487.0,Ethiopian Airlines,17:30,Seg,9,22 Ago 2022
3,1487.0,Ethiopian Airlines,17:30,Seg,9,29 Ago 2022
4,1487.0,Ethiopian Airlines,17:30,Qua,7,31 Ago 2022
